In [7]:
import json
import requests
import os
import pandas as pd

In [16]:
file = open('../key.json', 'r')
API_KEY = json.load(file)[0]['OMDB_API']

In [4]:
df = pd.read_csv("movies.csv")
movies = df.to_dict(orient="records")

In [18]:
movies[0]

{'Unnamed: 0': 0,
 'date': 'Jan 4, 2013',
 'title': 'Texas Chainsaw 3D',
 'production budget': '$20,000,000',
 'domestric budget': '$34,341,945',
 'worldwide gross': '$47,666,013'}

In [97]:
def contains_movie(movie_list: list, title: str):
    for m in movie_list:
        if len(m.keys())<3:
            return False
        if m["Title"].lower() == title.lower():
            return True
    return False

In [91]:
movies_info = []

In [161]:
for movie in movies:
    if len(movies_info)>0 and contains_movie(movies_info, movie['title']):
        continue

    movie = movie['title'].replace(" ", "+")
    request_url = f"http://www.omdbapi.com/?t={movie}&apikey={API_KEY}"

    response = requests.request("GET", request_url)
    if response.status_code != 200:
        print(f"server error {response.status_code}")
        break

    if json.loads(response.text)['Response'] == 'False':
        continue

    data = json.loads(response.text)
    movies_info.append(data)

server error 401


In [57]:
# movie = movies[20]['title'].replace(" ", "+")
# request_url = f"http://www.omdbapi.com/?t={movie}&apikey={API_KEY}"

# response = requests.request("GET", request_url)
# # (json.loads(response.text), (bool) (json.loads(response.text)['Response']))
# type(response.status_code)
movies_info[2].keys()

dict_keys(['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director', 'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster', 'Ratings', 'Metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'Type', 'DVD', 'BoxOffice', 'Production', 'Website', 'Response'])

In [163]:
filter_movies = [m for m in movies_info if len(m.keys()) >= 3]

In [165]:
df = pd.DataFrame(filter_movies)
df.to_csv("movie_info.csv")

In [164]:
len(movies_info), len(filter_movies)

(1143, 1143)